# Drug Utilization EDA
Explore the drug utilization CSV found at https://data.medicaid.gov/api/1/datastore/query/eec7fbe6-c4c4-5915-b3d0-be5828ef4e9d/0/download?format=csv

In [1]:
import pandas as pd
import os

The preferred route to get all the raw data is via the Makefile.\
The following cell is just a catch all to ensure the data is available for EDA.

In [2]:
path = 'raw_data/drug_utilization_2021.csv'
if not os.path.exists(path):
    import get_data
    get_data.download_drug_utiliztion(path)

In [3]:
df = pd.read_csv(path)

### What do we notice about the first 3 rows?

In [4]:
df.head(3)

,utilization_type,state,ndc,labeler_code,product_code,package_size,year,quarter,suppression_used,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
0,FFSU,AK,2143380,2,1433,80,2021,4,False,TRULICITY,544.0,222.0,220042.29,215557.09,4485.20
1,FFSU,AK,2143480,2,1434,80,2021,4,False,TRULICITY,706.0,275.0,286542.68,281194.56,5348.12
2,FFSU,AK,2143611,2,1436,11,2021,4,False,EMGALITY P,27.0,27.0,16648.52,16648.52,0.00


Product names are more human readable than ndc codes so let's see how many we're dealing with.

In [5]:
# let's look at a specific record
df[df.ndc==2143380]['product_name'].unique()

array(['TRULICITY '], dtype=object)

Padding can be kind of a pain.  Let's strip that out and see if that changes our unique count

In [6]:
# count of unique product_name before blanks stripped out
len(df['product_name'].unique())

15326

In [7]:
df['product_name'] = df['product_name'].str.strip()

# count of unique product_name before blanks stripped out - should be the same as above
len(df['product_name'].unique())

15325

Since no ill effects (before/after counts are the same), we'll include stripping the padding in our cleaning step (clean.py) as it will dealing with product name queries a little easier.

### General data set stats

In [8]:
df.describe()

,ndc,labeler_code,product_code,package_size,year,quarter,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
count,5.020759e+06,5.020759e+06,5.020759e+06,5.020759e+06,5020759.0,5.020759e+06,2.571171e+06,2.571171e+06,2.571171e+06,2.571171e+06,2.571171e+06
mean,3.754980e+10,3.754971e+04,1.386811e+03,2.232003e+01,2021.0,2.511108e+00,5.567704e+04,5.717950e+02,6.623537e+04,6.331987e+04,2.915500e+03
std,2.825187e+10,2.825197e+04,2.162050e+03,2.759782e+01,0.0,1.117977e+00,1.076842e+07,5.147971e+03,1.307438e+06,1.281386e+06,8.568518e+04
min,2.010102e+06,2.000000e+00,0.000000e+00,0.000000e+00,2021.0,1.000000e+00,6.000000e-03,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.811089e+08,7.810000e+02,1.850000e+02,1.000000e+00,2021.0,2.000000e+00,7.760000e+02,2.300000e+01,4.323300e+02,4.120700e+02,0.000000e+00
50%,4.778107e+10,4.778100e+04,5.000000e+02,1.000000e+01,2021.0,3.000000e+00,2.368000e+03,5.600000e+01,1.565070e+03,1.486290e+03,0.000000e+00
75%,6.467908e+10,6.467900e+04,1.079000e+03,3.100000e+01,2021.0,4.000000e+00,9.480000e+03,1.980000e+02,7.544500e+03,7.117820e+03,5.479000e+01
max,1.000000e+11,9.999900e+04,9.999000e+03,9.900000e+01,2021.0,4.000000e+00,1.028045e+10,1.234377e+06,5.185489e+08,5.140411e+08,3.627057e+07


The first thing I notice is while there are 5M rows, only about 1/2 of them have entries for: 
* units_reimbursed
* number_of_prescriptions
* total_amount_reimbursed
* medicaid_amount_reimbursed
* non_medicaid_amount_reimbursed

These seem to correlate with suppression_used as false.  To confirm, we'll perform the following two tests
1. Whenever suppression_used is true, all the above fields are na (no non na fields)
2. Whenever suppression_used is false, none of the above fields are na (no na fields)

##### 1. Whenever suppression_used is true, all the above fields are na (no non na fields)

In [9]:
# ensure count of suppression_used==True > 0
df[df['suppression_used']].shape[0]

2449588

In [10]:
# Find the intersection of suppression_used and any non-na values
# A count of 0 means no overlap between supression true and any other of specified fields having a non-na value

df[(df['suppression_used']) & 
   (
       (~df['number_of_prescriptions'].isna()) | 
       (~df['units_reimbursed'].isna()) | 
       (~df['total_amount_reimbursed'].isna()) |
       (~df['medicaid_amount_reimbursed'].isna()) |
       (~df['non_medicaid_amount_reimbursed'].isna()) 
)].shape[0]

0

##### 2. Whenever suppression_used is false, none of the above fields are na (no na fields)

In [11]:
# ensure count of suppression_used==False > 0
df[~df['suppression_used']].shape[0]

2571171

In [12]:
# Find the intersection of suppression_used==False and any na values
# A count of 0 means no overlap between supression false and any other of specified fields with value of na

df[(~df['suppression_used']) & 
   (
       (df['number_of_prescriptions'].isna()) | 
       (df['units_reimbursed'].isna()) | 
       (df['total_amount_reimbursed'].isna()) |
       (df['medicaid_amount_reimbursed'].isna()) |
       (df['non_medicaid_amount_reimbursed'].isna()) 
)].shape[0]

0

#### Conclusion
suppression_used can be used as a filter to determine if the above mentioned fields will have a non-na value